# CP 2 Aprendizaje de Máquinas
---
## Clasificación supervisada e introducción a la herramienta sklearn

El objetivo de la clase práctica es resolver un problema utilizando los algoritmos dados en conferencia. Se tienen críticas de cine positivas y negativas. 1000 de cada una. El objetivo es, dado el texto de la crítica, determinar si la crítica es positiva o negativa.

### Ejercicio 1: Extracción los datos

En la carpeta `txt_sentoken` se encuentran las críticas. Dentro hay dos carpetas (`pos`, `neg`), donde están las críticas positivas y negativas.

In [1]:
path_p = u"txt_sentoken/pos"
path_n = u"txt_sentoken/neg"

Cada archivo de cada carpeta es una crítica diferente, por lo que tenemos que analizar los archivos de cada una de estas carptetas para procesar las críticas.

In [2]:
import os
ds_p = os.listdir(path_p)
ds_n = os.listdir(path_n)

Luego, cada archivo tenemos que convertirlos en cadenas de texto.

In [3]:
def convert_file_to_text(name: str):
    text = u""
    f = open(name)
    for i in f.readlines():
        text += i
    return text
        

Por ejemplo, de esta forma se convierte un archivo:

In [4]:
a = convert_file_to_text(os.path.join(path_p, ds_p[0]))
a

'films adapted from comic books have had plenty of success , whether they\'re about superheroes ( batman , superman , spawn ) , or geared toward kids ( casper ) or the arthouse crowd ( ghost world ) , but there\'s never really been a comic book like from hell before . \nfor starters , it was created by alan moore ( and eddie campbell ) , who brought the medium to a whole new level in the mid \'80s with a 12-part series called the watchmen . \nto say moore and campbell thoroughly researched the subject of jack the ripper would be like saying michael jackson is starting to look a little odd . \nthe book ( or " graphic novel , " if you will ) is over 500 pages long and includes nearly 30 more that consist of nothing but footnotes . \nin other words , don\'t dismiss this film because of its source . \nif you can get past the whole comic book thing , you might find another stumbling block in from hell\'s directors , albert and allen hughes . \ngetting the hughes brothers to direct this seem

Luego, hacemos lo mismo para cada archivo, guardándolos en dos listas diferentes, una de críticas positivas y otra de críticas negativas.

In [5]:
texts_p = []
texts_n = []

In [6]:
for file in ds_p:
    texts_p.append(convert_file_to_text(os.path.join(path_p, file)))

In [7]:
for file in ds_n:
    texts_n.append(convert_file_to_text(os.path.join(path_n, file)))

Para comprobar el resultado de este proceso, se comprueba la longitud de las listas

In [8]:
len(texts_p)

1000

In [9]:
len(texts_n)

1000

### Ejercicio 2: Extracción de características

¿Cómo convertimos una cadena de texto en una matriz de características?

Existen varias maneras, una de las más sencillas, es usando `CountVectorizer` de `sklearn`, que convierte una collección de documentos en una matriz con recuentos de tokens.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer 

In [11]:
vectorizer = CountVectorizer()
mt = vectorizer.fit_transform(texts_p + texts_n)
mta = mt.toarray()

La matriz de características tiene dos dimensiones, la primera representa la cantidad de instancias (ejemplos) y la segunda la cantidad de características. 

In [12]:
# matriz de características
mt.shape

(2000, 39659)

¿Cuál es el por ciento de los valores que son distintos de cero? 

In [13]:
mt.nnz *100.0 / (mt.shape[0] * mt.shape[1])

0.8407196348874152

El vector de clase dice, para cada instancia cuál es la clase a la que pertenece. La clase positiva (1) representa una crítica positiva y la clase negativa (0) una crítica negativa.

In [14]:
y = [1]*1000 + [0]*1000

### Ejercicio 3: Elección del clasificador

Ahora que ya se tienen la matriz de clasificación y el vector de clases, ¿qué hacemos?, ¿qué clasificador usamos?, ¿por qué?

El Naive Bayes de sklearn se llama Gaussian Naive Bayes porque está diseñado (a diferencia del visto en conferencia) para lidiar con características que sean valores continuos. En vez de tener probabilidad del feature dada la clase que es $P(w|c)$ ahora lo que se tiene
es la probabilidad de observar un valor tan alto como $x_i$, esto es $P(x \leq x_i | c)$


In [15]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

### Ejercicio 4: Entrenamiento

Para comenzar el entrenamiento del clasificador elegido, es necesario dividir la matriz de características y el vector de clase en dos conjuntos, uno para entrenamiento y otro de prueba.

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(mta, y, train_size=0.60)

In [17]:
print(X_train.shape)
print(X_test.shape)

(1200, 39659)
(800, 39659)


In [18]:
naive_bayes = GaussianNB()
naive_bayes.fit(mta,y)
naive_bayes.score(mta,y)

0.9985

In [19]:
naive_bayes = GaussianNB()
naive_bayes.fit(X_train, y_train)
naive_bayes.score(X_test, y_test)

0.65875

In [20]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.score(X_test, y_test)

0.62875

In [21]:
from scipy.stats import ttest_ind

In [22]:
#se hacen 30 corridas
rs = []
for i in range(30):
    X_train, X_test, y_train, y_test = train_test_split(mta, y, train_size=0.60)
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    rs.append(knn.score(X_test, y_test))

In [23]:
rs

[0.61,
 0.6275,
 0.615,
 0.59625,
 0.64375,
 0.6225,
 0.62125,
 0.62875,
 0.6025,
 0.64,
 0.59375,
 0.63375,
 0.625,
 0.6275,
 0.61625,
 0.6175,
 0.62875,
 0.6125,
 0.645,
 0.64,
 0.62875,
 0.645,
 0.64375,
 0.6175,
 0.605,
 0.5975,
 0.6525,
 0.625,
 0.61,
 0.60625]

In [27]:
import numpy as np
np.mean(rs)

0.6226249999999999

In [25]:
=

SyntaxError: invalid syntax (<ipython-input-25-9ae22869553b>, line 1)